# Machine, Data & Learning

## Project | Genetic Algorithm

Team 30 | Tribrid
-----------
* Nitin Chandak (2019101024)
* Ayush Sharma (2019101004)

In [196]:
# Usefull Imports
import client as server
import numpy as np
import random
import json
import itertools
import os

### Default Initial Overfit Vector
```
[
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
```

In [197]:
# Usefull Global Constants & lambda functions

TEAM_ID = 'colthAUIKUTfdh4qWrnHBhJzkyEm8kt4qIue1BKtyvLItfp8Po'

DEFAULT_INITIAL_OVERFIT_VECTOR = [
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
MY_INITIAL_VECTOR = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
POPULATION_SIZE = 10 # intentionaly chose this as we have to store 10 best vectors
GENERATION_LOOP = 10
MIN_GENE_VAL = -10
MAX_GENE_VAL = 10
MUTATION_PROBABILITY = 0.5
MUTATION_FACTOR = lambda : random.uniform(0.9, 1.1)
MATING_POOL_SIZE = 6

OUTPUT_FILE = 'output1.txt'

In [198]:
def is_output_file(fileName):
    '''
    This funtion returns False if 
    filename exist  otherwise True
    '''
    return os.path.exists(fileName)

In [199]:
def is_valid_file(filename):
    '''
    This function checks existence of file
    filename and if exist then checks if it
    empty or not.
    '''
    if is_output_file(filename):
        return (os.stat(filename).st_size != 0)
    else:
        return False

In [201]:
def create_default_output_file():
    '''
    This function will create a default
    output file for dumping our 10 best vector
    before any iteration of algorithm when called.
    '''
    answer = []
    for i in range(10):
        ret_vector = MY_INITIAL_VECTOR.copy()
        for j in range(len(ret_vector)):
            if random.randint(1,10)<(10*MUTATION_PROBABILITY):
                new_gene = DEFAULT_INITIAL_OVERFIT_VECTOR[j]*MUTATION_FACTOR()
                if abs(new_gene)<10:
                    ret_vector[j]=new_gene
                else:
                    ret_vector[j]=random.uniform(-1,1)
        print('i = ',i,"\n curr => \n",ret_vector)
        answer.append(ret_vector)
    print("Answer =>\n",answer)
    with open(OUTPUT_FILE,'w') as write_file:
        json.dump(answer, write_file)

In [202]:
def read_output_file():
    '''
    This function will read the OUTPUT_FILE
    and return the answer array there which 
    contains 10 best vector obtained so far.
    '''
    with open(OUTPUT_FILE,'r') as write_file:
        answer=json.load(write_file)
    return answer

In [203]:
def dump_best_vectors(answer: list):
    '''
    This function will dump the list of
    10 vectors into the OUTPUT_FILE file.
    And return the updated data.
    '''
    with open(OUTPUT_FILE,'w') as write_file:
        json.dump(answer, write_file)
    return read_output_file()

In [204]:
#@TODO: Within While loop:
#       Generate new Population
#       Get errors, fitness & curr_weight_distribution
#       Update 10 best vector array i.e. make a separate list for it.
#       Keep track of generations of the vecotrs for generation folder.
#       Dump 10 best vectors into output.txt file.

In [207]:
def get_both_err(population):
    '''
    This function utilises the API 
    call provided to us for getting the
    errors on the vectors within the population.
    
    Parameter
    ---------
    population: list of vector of 11-D
    
    Return
    ------
    It returns two list train_err & validation_err
    which are errors for the given poplation's vectors.
    '''
#     train_err = [ random.randint(1,300) for i in range(len(population))]
#     validation_err = [ random.randint(1,300) for i in range(len(population))]
    
    train_err = []
    validation_err = []
    for individual in population:
        [te, ve]= server.get_errors(TEAM_ID,individual)
        train_err.append(te)
        validation_err.append(ve)
    return train_err, validation_err

In [208]:
def get_cost(train_error, validation_error):
    '''
    This function calculates the cost
    for given list of errors. Lower the
    cost more fit/perfect the vector.
    
    Parameter
    ---------
    Requires two list train_err & validation_err
    which are errors for the given poplation's vectors.
    
    Return
    ------
    Returns the list of fitness for corresponding errors.
    '''
    cost = []
    for i in range(len(train_error)):
        sum_err = train_error[i] + validation_error[i]
        abs_diff_err = abs(train_error[i] - validation_error[i])
        fit = sum_err + 2 * abs_diff_err
        cost.append(fit)
    return cost

In [209]:
def get_initial_population_info():
    '''
    This function will return a stack
    of all the 10 best vector along with
    their Train error, Validation error
    and their cost
    '''
    initial_population = read_output_file()
    train_error, validation_error = get_both_err(initial_population)
    initial_cost = get_cost(train_error, validation_error)
    initial_population_info = np.column_stack((initial_population,train_error, validation_error,initial_cost))
    return initial_population_info

In [210]:
def create_mating_population(parent_population):
    '''
    This function will select top MATING_POOL_SIZE
    individuals with least costs from the parent_population 
    for cross-over.
    '''
    mating_pool = parent_population[:MATING_POOL_SIZE]
    return mating_pool    

In [211]:
def cross_over(p1,p2):
    '''
    This function simply does the cross-over
    on two individual p1,p2 and returns c1,c2
    i.e. crossed-child.
    '''
    crossover_point = random.randint(1, 10)
    c1 = list(p1[:crossover_point]) + list(p2[crossover_point:])
    c2 = list(p2[:crossover_point]) + list(p1[crossover_point:])
    return c1, c2

def cool_cross_over(p1,p2):
    c1 = np.empty(11)
    c2 = np.empty(11)

    u = random.random() 
    n_c = 3
        
    if (u < 0.5):
        beta = (2 * u)**((n_c + 1)**-1)
    else:
        beta = ((2*(1-u))**-1)**((n_c + 1)**-1)


    p1 = np.array(p1)
    p2 = np.array(p2)
    c1 = 0.5*((1 + beta) * p1 + (1 - beta) * p2)
    c2 = 0.5*((1 - beta) * p1 + (1 + beta) * p2)

    return c1.tolist(), c2.tolist()


def simulate_cross_over(mating_population):
    '''
    This function will perform the cross-over
    on mating_population and generate POPULATION_SIZE
    total childs/individual.
    '''
    mating_population = mating_population[:,:-3]
    crossed_population = []
    for i in range(POPULATION_SIZE//2):
        p1 = mating_population[random.randint(0,MATING_POOL_SIZE-1)]
        p2 = mating_population[random.randint(0,MATING_POOL_SIZE-1)]
        c1, c2 = cross_over(p1,p2)
        crossed_population.append(c1)
        crossed_population.append(c2)
    return crossed_population

In [212]:
def mutate_population(crossed_population):
    '''
    This function will mutate the crossed_population
    and returns the mutated population.
    '''
    mutated_population = []
    for i in range(len(crossed_population)):
        curr_population = crossed_population[i]
        for j in range(len(curr_population)):
            if random.randint(1, 10) <= (10*MUTATION_PROBABILITY):
                new_gene = curr_population[j]*MUTATION_FACTOR()
                if abs(new_gene)<=10:
                    curr_population[j]=new_gene
        mutated_population.append(curr_population)
    return mutated_population

In [213]:
def create_next_generation(mutated_population,parent_population_info):
    '''
    This funtion will get the errors & cost for the mutated population
    then sort it in increasing order. The next_generation will be the 
    combination of 3 top parent & (POPULATION_SIZE-3) mutated childs.
    '''
    train_err, validation_err = get_both_err(mutated_population)
    mutated_cost = get_cost(train_err, validation_err)
    mutated_population_info = np.column_stack((mutated_population,train_err, validation_err,mutated_cost))
    mutated_population_info = mutated_population_info[np.argsort(mutated_population_info[:,-1])]
    
    select_parent = parent_population_info[:3]
    select_mutated = mutated_population_info[:(POPULATION_SIZE-3)]
    
    next_generation = np.concatenate((select_parent,select_mutated))
    next_generation = next_generation[np.argsort(next_generation[:,-1])]
    return next_generation

In [214]:
def submit_vector(vector: list):
    '''
    This function has been written for submitting the best
    vector generated so far for intermediate evaluation.
    `vector` is a list of length 14 with last three
    data as train_err, validation_err & cost.
    '''
    print(vector)
    submit_vector = vector[:-3]
    response = server.submit(TEAM_ID,submit_vector.tolist())
    print(response)

In [215]:
def update_best_vector_set(next_generation,best_vector_set):
    '''
    This function will merge next_generation 
    & best_vector_set in increasing order of cost.
    Then return the top 10 vectors from it.
    '''
    print('Next generation : \n',next_generation)
    print('best_vector_set_yo : \n',best_vector_set)
    new_best_vector_set = np.concatenate((next_generation,best_vector_set))
    k = new_best_vector_set[np.argsort(new_best_vector_set[:,-1])].tolist()
    new_best_vector_set = [k for k,_ in itertools.groupby(k)]
    new_best_vector_set = np.array(new_best_vector_set)
    return new_best_vector_set[:10]

In [216]:
def dump_all_generation(all_generation):
    '''
    '''
    if not is_valid_file('generation.txt'):
        with open('generation.txt','w') as write_file:
            previous_generations = []
            previous_generations.append(all_generation)
            json.dump(previous_generations, write_file)    
    else:
        with open('generation.txt','r+') as write_file:
            previous_generations = json.load(write_file)
            previous_generations.append(all_generation)
            json.dump(previous_generations, write_file)

In [217]:
def GA():
    '''
    Main function to be called to run
    the implemented genetic algorithm.
    '''
    if not is_valid_file(OUTPUT_FILE):
        create_default_output_file()
        print('created a new output.txt')
    
    all_generation = []       
    initial_population_info = get_initial_population_info()
    parent_population = initial_population_info[np.argsort(initial_population_info[:,-1])]
    best_vector_set = parent_population
    
    for generation in range(GENERATION_LOOP-1):
        
        mating_population = create_mating_population(parent_population)
        crossed_population = simulate_cross_over(mating_population)
        mutated_population = mutate_population(crossed_population)
        next_generation = create_next_generation(mutated_population,parent_population)
        best_vector_set = update_best_vector_set(next_generation,best_vector_set)
        
        reproduction_info_dict = {
            'parent_population': parent_population[:,:-3].tolist(),
            'error': parent_population[:,-3:-1].tolist(),
            'cost': parent_population[:,-1:].tolist(),
            'mating_population' : mating_population.tolist(),
            'crossed_population' : np.array(crossed_population).tolist(),
            'mutated_population' : np.array(mutated_population).tolist(),
            'next_generation' : next_generation[:,:-3].tolist(),
            'next_error': next_generation[:,-3:-1].tolist(),
            'next_cost': next_generation[:,-1:].tolist()
        }
        all_generation.append(reproduction_info_dict)
        parent_population = next_generation
    return best_vector_set, all_generation

In [224]:
best_vector_set, all_generation = GA()
print("Final=>\n",best_vector_set)
print("Length of best vectors set =>",len(best_vector_set))
# dump_all_generation(all_generation)

Next generation : 
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.80710009e-10  4.23544351e+11
   4.46383148e+11  9.15605093e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.04687685e-15  2.14139543e-05
  -1.86735394e-06 -2.10849818e-08  9.93281914e-10  4.74714415e+11
   4.70729874e+11  9.53413372e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.14139543e-05
  -1.86735394e-06 -2.10849818e-08  9.93281914e-10  4.74714415e+11
   4.70729874e+11  9.53413372e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.93281914e-10  4.45355846e+11
   4.66739609e+11  9.54862980e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

Next generation : 
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.32175860e-05
  -1.86735394e-06 -2.10849818e-08  9.71900597e-10  3.65704853e+11
   4.06314219e+11  8.53237805e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.80710009e-10  4.23544351e+11
   4.46383148e+11  9.15605093e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.03665224e-15  2.14139543e-05
  -1.86735394e-06 -2.10778699e-08  9.93281914e-10  4.74441431e+11
   4.70967731e+11  9.52356563e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.04687685e-15  2.14139543e-05
  -1.86735394e-06 -2.10849818e-08  9.93281914e-10  4.74714415e+11
   4.70729874e+11  9.53413372e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

Next generation : 
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.32175860e-05
  -1.86735394e-06 -2.10849818e-08  9.71900597e-10  3.65704853e+11
   4.06314219e+11  8.53237805e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.83457814e-16  2.24222563e-05
  -1.86735394e-06 -2.06320444e-08  9.71900597e-10  3.69626044e+11
   4.16109999e+11  8.78703954e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.80710009e-10  4.23544351e+11
   4.46383148e+11  9.15605093e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.03665224e-15  2.14139543e-05
  -1.86735394e-06 -2.10778699e-08  9.93281914e-10  4.74441431e+11
   4.70967731e+11  9.52356563e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

Next generation : 
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.65515986e-16  2.36790645e-05
  -1.85020126e-06 -2.14591291e-08  9.71900597e-10  3.66922756e+11
   3.67996568e+11  7.37066948e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.32175860e-05
  -1.86735394e-06 -2.10849818e-08  9.71900597e-10  3.65704853e+11
   4.06314219e+11  8.53237805e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.83457814e-16  2.24222563e-05
  -1.86735394e-06 -2.06320444e-08  9.71900597e-10  3.69626044e+11
   4.16109999e+11  8.78703954e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.80710009e-10  4.23544351e+11
   4.46383148e+11  9.15605093e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

Next generation : 
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.65515986e-16  2.36790645e-05
  -1.85020126e-06 -2.14591291e-08  9.71900597e-10  3.66922756e+11
   3.67996568e+11  7.37066948e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.45517495e-05
  -1.86735394e-06 -2.06320444e-08  9.54494847e-10  2.63080517e+11
   3.34869223e+11  7.41527151e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.14761029e-15  2.32175860e-05
  -1.86735394e-06 -2.10849818e-08  9.71900597e-10  3.65704853e+11
   4.06314219e+11  8.53237805e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.21986871e-15  2.33851715e-05
  -1.70190202e-06 -2.10849818e-08  8.80852469e-10  5.93743750e+11
   6.35271819e+11  1.31207171e+12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

In [225]:
print(np.array(read_output_file()))
dump_data = best_vector_set[:,:-3]
print(dump_data)
dump_best_vectors(dump_data.tolist())

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.80710009e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.04687685e-15  2.14139543e-05
  -1.86735394e-06 -2.10849818e-08  9.93281914e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.27485799e-05
  -1.86735394e-06 -2.13925334e-08  9.93281914e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01250839e-15  2.20926514e-05
  -1.91631159e-06 -2.03373272e-08  9.93281914e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00747376e-15  2.20926514e-05
  -1.91631159e-06 -2.03373272e-08  9.93281914e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  9.65515985895032e-16,
  2.367906448336153e-05,
  -1.8502012568751136e-06,
  -2.145912905015716e-08,
  9.719005968953905e-10],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.1476102851656589e-15,
  2.45517494530711e-05,
  -1.8673539410909739e-06,
  -2.063204443806975e-08,
  9.544948471220997e-10],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.1476102851656589e-15,
  2.3217586009396077e-05,
  -1.8673539410909739e-06,
  -2.1084981760954167e-08,
  9.719005968953905e-10],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  9.834578138644201e-16,
  2.2422256259325252e-05,
  -1.8673539410909739e-06,
  -2.063204443806975e-08,
  9.719005968953905e-10],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0125083905414464e-15,
  2.274857993271698e-05,
  -1.8673539410909739e-06,
  -2.139253342360335e-08,
  9.807100090638911e-10],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.044330540344263e-15,
  2.367906448336153e-05,
  -1.7708786252882768e-06,
  -2.145912905015716e-

In [227]:
submit_vector(best_vector_set[0])
# submit_v = [ 0.00000000e+00, -1.32224490e-12, -2.62445101e-13,  4.00682289e-11,
#  -1.76372543e-10, -1.84826898e-15,  9.51748802e-16,  2.14682832e-05,
#  -2.21274832e-06, -1.49786697e-08,  8.00970477e-10,] 
# print(server.submit(TEAM_ID,submit_v))

# submit_v = [0,0,0,0,0,0,0,0,0,0,0] 
# print(server.submit(TEAM_ID,submit_v))

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.65515986e-16  2.36790645e-05
 -1.85020126e-06 -2.14591291e-08  9.71900597e-10  3.66922756e+11
  3.67996568e+11  7.37066948e+11]
authentication/usage quota error
